<br> Що ж. Для кінцевого результату нам важливо декілька речей
 - Те, як правильно модель буде розуміти поставлені задачі (bufix, enhancement, feature, etc.) 
 - Планування - як добре модель може розписати покрокове рішення задачі 
 - Написання коду - наскільки він валідний 

<br> Щоб досягти цього  існують декілька варіантів:
 1. запхати весь код проєкту у модель та сподіватися на краще 
 2. розібрати декілька іш'юсів з покроковим поясненням кожної дії 
або ж обидва пункти одразу 

<br> Що важливо? 
1. Якщо модель не знатиме вже написаного коду, вона не зможе давати точні відповіді та вказувати на конкретні помилки / виправлення. Таким чином її функціонал обмежується до абстрактних порад на основі вже вивченого з інтернету коду. 
Це проблема, яку я намагаюсь вирішити, оскільки такий ефект рівносильний тому, щоб запхати код у чатгпт без будь-яких додаткових дій. 

2. У більшості випадків, модель так чи інакше не здатна вигадати геніальне рішення для нової фічі. ( хоча шаблонні рішення повинні даватись легко )
З цим нічого не зробиш, оскільки у людського розробника досвіду куди більше, та людина врешті-решт може креативно мислити та комунікувати.

Перед тим, як переходити до створення датасету, давайте проведемо невеличке дослідження, все ж, ми тут датасаєнсом займаємось! <br>
З [треду](https://www.reddit.com/r/ChatGPT/comments/11ax0cy/is_it_possible_to_give_chatgpt_whole_of_my/) на реддіті, який чудово описує перше питання яке постало перед нами. 
<br>У мене вийшло дістати трошки корисної інформації, а саме те, що велика кількість коду не вміститься у короткостроковій пам'яті гпт, для цього існуює embedingsAPI
<br>А також та знайшов [cookbook](https://cookbook.openai.com/examples/get_embeddings_from_dataset) яким демонструє як отримати embeddings з датасету 

Що ж таке ці embeddings, та з чого вони зліплені? [opeanAI](https://platform.openai.com/docs/guides/embeddings#:~:text=OpenAI's%20text%20embeddings%20measure%20the,related%20text%20strings%20are%20recommended)
<br> Одразу відповідаючи на це питання: це векторна репрезентація слів, де вектор не тільки позначає саме по собі слово, а також його важливість та як воно відноситься до інших слів. 
<br> Те, як слова відносяться одне до одного визначити просто: це відстань між їх векторами.

А тепер давайте спробуємо! 
<br> Для наглядності та економії часу у прикладі я буду використовувати реальний код проєкту.

Спочатку імпортуємо всі необхідні бібліотеки 

In [55]:

import os
import sys
import subprocess
import shutil
import pandas as pd
import tiktoken
#from utils.embeddings_utils import get_embedding
from dotenv import load_dotenv
from openai import OpenAI
import openai
load_dotenv('../../.env')

True

Визначемо шлях завантаження, посилання на сам репозиторій та файл який ми будемо використовувати для прикладу 
<br> Зокрема зазначимо модель, яку будемо використовувати 

In [56]:
URL = "https://github.com/fictadvisor/fictadvisor-web.git" 
PATH = '../../../assets/repositories/frontend/'
FILE = 'src/components/pages/personal-teacher-page/PersonalTeacherPage.tsx'
MODEL =  "text-embedding-ada-002"
GPTMODEL = "gpt-3.5-turbo"

In [57]:
to_clone = False # Щоб не клонувати кожен раз репозиторій, коли він вже є, 
if to_clone:
    directory = '../assets/repositories/frontend/'
    if os.path.exists(directory):
        shutil.rmtree(directory)
    subprocess.run(["git", "clone", URL, PATH])

Прочитаємо наш файл

<br> Оскільки ми працюємо з трансформером, кожне слово повинно бути перетворене на токен ( у нашому випадку, у вигляді вектору ), оскільки мережі не розуміють слів напряму. 
<br> Існує багато способів токенізації, але зараз ми просто використаємо той, що пропонують самі openAI. 
<br> Чому це для нас взагалі важливо? 
<br> У кожної з запропонованих моделей на сьогоднішній день існує максимальна кількість токенів, що є для нас проблемою.
<br> Але не тільки у цьому справа: за велику кількість токенів доведеться і більше платити :>

<br> Годі слів, час пробувати!

Для початку відкриємо не великий за змістом файл, скоріше серендій з оглядкою на мій досвід.

In [84]:
with open(PATH + FILE, 'r') as file:
    filedata = file.read()
print(filedata)

import {
  createContext,
  Dispatch,
  FC,
  SetStateAction,
  useEffect,
  useState,
} from 'react';
import { useRouter } from 'next/router';

import Breadcrumbs from '@/components/common/ui/breadcrumbs';
import PersonalTeacherCard from '@/components/common/ui/cards/personal-teacher-card';
import Progress from '@/components/common/ui/progress';
import PersonalTeacherTabs from '@/components/pages/personal-teacher-page/personal-teacher-tabs';
import styles from '@/components/pages/personal-teacher-page/PersonalTeacherPage.module.scss';
import {
  PersonalTeacherPageProps,
  TeachersPageTabs,
} from '@/components/pages/personal-teacher-page/utils';
import useTabState from '@/hooks/use-tab-state';
import useToast from '@/hooks/use-toast';
import { Teacher } from '@/types/teacher';

// TODO: move context to separate folder, move types to separate folder
export interface TeacherContext {
  floatingCardShowed: boolean;
  setFloatingCardShowed: Dispatch<SetStateAction<boolean>>;
  teacher: T

Та порахуємо скільки буде коштувати запхати його у нашу модель! ( багато ) ( бюджет СР не резиновий )
<br> Для початку завантажимо токенайзер для нашої моделі

In [67]:
client = OpenAI()
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
# Downloading encodings for our model 
encoding = tiktoken.encoding_for_models(MODEL)

AttributeError: module 'tiktoken' has no attribute 'encoding_for_models'

In [ ]:
def count_tokens(text):
    return len(encoding.encode(text))

count_tokens(filedata)

744

744 Токени це велике число, не підйомне для мого гаманця, враховуючи що на проєкті fictadvisor більш ніж 950 файлів! 
<br> Давайде дізнаємось його ціну, можливо все не так погано

In [ ]:
def calculate_price(tokens):
    """ 
    Model            Training                Input usage             Output usage
    gpt-3.5-turbo    $0.0080 / 1K tokens	 $0.0030 / 1K tokens     $0.0060 / 1K tokens
    """
    return tokens/1000*0.0060

print(f"{count_tokens(filedata)} tokens for {calculate_price(count_tokens(filedata))}$")

744 tokens for 0.004464$


Отже, багатенько, як на один файл. Пропоную порахувати скільки коштувало б скормити весь код fictadvisor frontend нашій моделі! 

In [ ]:
def tokens_in_each_file(directory):
    total_tokens = 0
    num_files = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                filedata = file.read()
            tokens = count_tokens(filedata)
            total_tokens += tokens
            num_files += 1
    return  {'total_tokens': total_tokens, 'total_files': num_files} 


data = tokens_in_each_file(PATH) 
print(
    f"Total tokens: {data['total_tokens']}" 
    + f"\n Total amount of files: {data['total_files']}" 
    + f"\n Average tokens per file: {data['total_tokens'] / data['total_files'] }"
    + f"\n Cost for all files: {calculate_price(data['total_tokens'])}$"
    + f"\n Cost per file: {calculate_price(data['total_tokens']) / data['total_files']}$"
)

Total tokens: 36302884
 total amount of files: 1024
 Average per file: 2.8207125362271494e-05
 Cost for all files: 217.817304$
 Cost per file: 0.2127122109375$


217 Доларів, що ж, я не можу собі дозволити таке задоволення. Треба трошки подумати!
<br> Перш за все, варто зауважити, що наведений код бере абсолютно всі файли які є у проєкті, це можуть бути і картинки також, а картинки можуть займати дуже багато місця
<br> Окрім  того, код не прибирає імпорти, які є у кожному файлі, і їх на справді дуже багато. Для тренування це не така важлива інформація, а ось грошей тягне багато. 
<br> Далі: пробіли, код не розділяється на слова, тому зайві пробіли можуть бути токенізовані, цього теж слід уникати 
<br> Тому давайте перерахуємо вартість!

In [ ]:
def calculate_average_tokens_per_file(directory):
    total_tokens = 0
    num_files = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            # skip non-js/ts files
            if not (filename.endswith('.js') or filename.endswith('.jsx') or filename.endswith('.ts') or filename.endswith('.tsx')):
                continue
            
            file_path = os.path.join(dirpath, filename)
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                filedata = file.readlines()
            # skip imports and delete unnecessary spaces
            
            filedata = ''.join([line for line in filedata.split("\n") if 'import' not in line])
            filedata = ' '.join(word.strip() for line in filedata.split() for word in line.split())
            
            tokens = count_tokens(filedata)
            total_tokens += tokens
            num_files += 1
    return {'total_tokens': total_tokens, 'total_files': num_files}

data = calculate_average_tokens_per_file(PATH) 
print(
    f"Total tokens: {data['total_tokens']}" 
    + f"\n Total amount of files: {data['total_files']}" 
    + f"\n Average tokens per file: {data['total_tokens'] / data['total_files'] }"
    + f"\n Cost for all files: {calculate_price(data['total_tokens'])}$"
    + f"\n Cost per file: {calculate_price(data['total_tokens']) / data['total_files']}$"
)

Total tokens: 332705
 Total amount of files: 897
 Average tokens per file: 370.9085841694537
 Cost for all files: 1.99623$
 Cost per file: 0.0022254515050167223$


Що ж, добре, той факт, що ми не розоримося на цьому проєкті не може не радувати.  
<br> Давайте все-ж таки спробуємо зробити з цього embedding

Для порівняння, попросимо зробити це завдання нашу модель до її файнтюнінгу

In [65]:
query = "how do i do PersonalTeacherPage using React, Next, MUI for my website?"
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": ""},
    {"role": "user", "content": f"{query}"}
  ]
)

In [64]:
print(completion.choices[0].message.content)  

To create a personal teacher page for your website, follow these steps:

1. Determine the purpose and content: Decide what information you want to include on your personal teacher page. Consider sharing your teaching philosophy, qualifications, experience, and any additional details that showcase your expertise and personality.

2. Choose a website builder: Select a website builder or content management system (CMS) that suits your needs. Popular options include WordPress, Wix, Weebly, and Squarespace. Look for templates or themes that are suitable for personal portfolios or professional profiles.

3. Register a domain name: Choose a domain name that reflects your name or teaching brand. Purchase your domain name from a domain registrar, such as GoDaddy or Namecheap.

4. Set up hosting: Activate website hosting or choose a platform that offers hosting services. Many website builders provide built-in hosting, simplifying the process.

5. Customize your website: Select a visually pleasin

Щож, очікувано, що він не знає відповіді на це питання
<br> Саме час це виправити!

In [98]:
import json

optimized = filedata
optimized = ''.join([line for line in optimized.split("\n") if 'import' not in line])
optimized = ' '.join(word.strip() for line in optimized.split() for word in line.split())

sample = {"messages": 
[
    {"role": "system", "content": ""}, 
    {"role": "user", "content": 'how do i do PersonalTeacherPage using React, Next, MUI for my website? Assume that i alrready have all needed components'},
    {"role": 'assistant', 'content': optimized}
]}

with open('sample.json', 'w') as outfile:
    json.dump(sample, outfile)

dataset_filepath = 'sample.json'

In [99]:

file = client.files.create(
  file=open("sample.json", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-WNDMetlwt4l0wuD14QRZ9PwH', bytes=2554, created_at=1702586840, filename='sample.json', object='file', purpose='fine-tune', status='processed', status_details=None)

In [108]:
suffix_name = "fictadvisor-advisor-proof-of-concept"
client.fine_tuning.jobs.create(
  training_file='', 
  model="gpt-3.5-turbo"
)

BadRequestError: Error code: 400 - {'error': {'message': 'invalid training_file: WNDMetlwt4l0wuD14QRZ9PwH', 'type': 'invalid_request_error', 'param': 'training_file', 'code': None}}